## 1. 데이터 전처리

In [2]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

빠른 학습을 위해 학습 데이터와 테스트 데이터를 2:8 비율로 분할합니다. 이 값을 변경하고자 하는 분은 test_ratio의 값을 변경하세요.



In [4]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

# 2. system_message 정의
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

# 3. 원본 데이터의 type별 분포 출력 
print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

# 4. train/test 분할 비율 설정 (0.5면 5:5로 분할)
test_ratio = 0.8

train_data = []
test_data = []

# 5. type별로 순회하면서 train/test 데이터 분할
for type_name in set(dataset['type']):
    # 현재 type에 해당하는 데이터의 인덱스만 추출
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]
    
    # test_ratio에 따라 test 데이터 개수 계산 
    test_size = int(len(curr_type_data) * test_ratio)
    
    # 현재 type의 데이터를 test_ratio 비율로 분할하여 추가
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

# 6. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # 검색 결과를 문서1, 문서2... 형태로 포매팅
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])
    
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

# 7. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 8. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

# 9. 분할된 데이터의 type별 분포 출력
print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

README.md:   0%|          | 0.00/909 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

원본 데이터의 type 분포:
no_answer: 404
mrc_question_with_1_to_4_negative: 296
paraphrased_question: 196
mrc_question: 491
synthetic_question: 497

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
no_answer: 81
mrc_question_with_1_to_4_negative: 60
paraphrased_question: 40
mrc_question: 99
synthetic_question: 100

테스트 데이터의 type 분포:
no_answer: 323
mrc_question_with_1_to_4_negative: 236
paraphrased_question: 156
mrc_question: 392
synthetic_question: 397


In [5]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 정부가 내용이 분명치 않거나 표현이 어색한 중앙행정기관의 영문 명칭을 일제히 정비해 법제화한다. 국제사회에서 부처 간 교류가 활발해지는 가운데 현 부처 영문 명칭을 외국인들이 이해하기 어렵다는 이유에서다.행정자치부는 “이 같은 내용을 담은 ‘정부조직 영어명칭에 관한 규칙’(예규)을 이르면 이달 중 제정할 계획”이라고 5일 밝혔다. 소속기관의 영문 명칭이 법제화되는 것은 1948년 대한민국 정부가 출범한 이래 처음이다.중앙부처 중 정비 대상 1순위는 기획재정부다. 전문가들은 현 ‘Ministry of Strategy and Finance’인 기재부 영문 명칭에서 ‘Strategy’를 빼야 한다고 지적한다. 국가의 미래 경제전략을 수립하는 부처라는 점을 강조하기 위해 쓰였지만 이를 보고 기재부를 떠올리는 외국인은 거의 없다는 게 전문가들의 지적이다. 행자부의 영문 명칭도 바뀔 전망이다. 현 명칭인 ‘Ministry of Government Administration and Home Affairs’는 정부 조직과 사무를 맡는다는 뜻이지만 

In [6]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [7]:
train_dataset[0]

{'messages': [{'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 콘코바르는 결국 죽게 되는데, 그 곡절은 이러하다. 라긴의 왕 메스 게그러의 뇌를 굳힌 겻을 울라의 코날이 전리품으로 가지고 있었는데, 코나크타의 전사 케트 막 마가크가 이를 훔쳐갔다. 그리고 케트는 무릿매로 메스 게그러의 뇌를 던져 콘코바르의 머리를 맞추었고, 메스 게그러의 뇌가 콘코바르의 머리통 깊숙히 박혀 버렸다. 이 일이 일어난 곳은 우르카르(Urchair)의 발러 아흐(Baile Ath), 곧 오늘날의 웨스트미스 주 호르셀리프라고 한다. 콘코바르의 의사들은 이 이물질을 제거할 수 없었고, 상처를 봉합만 한 뒤 왕에게 흥분하지 않으면 생명을 유지할 수 있다고 말했다. 7년이 평화롭게 흘러간 뒤 콘코바르는 그리스도가 죽었다는 소식을 듣게 되어 분노했고, 뇌가 터져 죽었다. 머리가 터진 자리에서 뿜어져나온 피의 세례를 받은 결과 그는 기독교인이 되었고 그 영혼은 천국으로 갔다. 콘코바르의 죽음에 관한 이 기록은 매우 얄팍한 기독교화가 이루어져 있는데, 한편 노르드 신화의 토르가 흐룽그니르와 싸우다 머리에 숫돌이 박힌 이야기와 유사한 점이 있다

In [10]:
# 데이터셋 저장
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1504 [00:00<?, ? examples/s]

## 2. 모델 로드 및 템플릿 적용


In [8]:
# 허깅페이스 모델 ID
model_id = "NCSOFT/Llama-VARCO-8B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

In [9]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 콘코바르는 결국 죽게 되는데, 그 곡절은 이러하다. 라긴의 왕 메스 게그러의 뇌를 굳힌 겻을 울라의 코날이 전리품으로 가지고 있었는데, 코나크타의 전사 케트 막 마가크가 이를 훔쳐갔다. 그리고 케트는 무릿매로 메스 게그러의 뇌를 던져 콘코바르의 머리를 맞추었고, 메스 게그러의 뇌가 콘코바르의 머리통 깊숙히 박혀 버렸다. 이 일이 일어난 곳은 우르카르(Urchair)의 발러 아흐(Baile Ath), 곧 오늘날의 웨스트미스 주 호르셀리프라고 한다. 콘코바르의 의사들은 이 이물질을 제거할 수 없었고, 상처를 봉합만 한 뒤 왕에게 흥분하지 않으면 생명을 유지할 수 있다고 말했다. 7년이 평화롭게 흘러간 뒤 콘코바르는 그리스도가 죽었다는 소식을 듣게 되어 분노했고, 뇌가 터져 죽었다. 머리가 터진 자리에서 뿜어져나온 피의 세례를 받은 결과 그는 기독교인이 되었고 그 영혼은 천국으로 갔다. 콘코바르의 죽음에 관한 이 기록은 매우 얄팍한 기독교화가 이루어져 있는데, 한편 노르드 신화의 토르가 흐룽그니르와 싸우다 머리에

## 3. LoRA와 SFTConfig 설정


In [11]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

- lora_alpha: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.
- lora_dropout: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화.
- r: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.
- bias: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.
- target_modules: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.
- task_type: LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [12]:
args = SFTConfig(
    output_dir="llama-3-8b-rag-ko",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

- `output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

- `num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니.

- `per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있니다.

- `gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 `2`로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 용합니다.

- `gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질수 있습니다.

- `optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적기를 사용합니다.

- `logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마 로그를 기록합니다.

- `save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스마다 모델이 저장됩니다.

- `save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50스텝마다 모델을 저장합니다.

- `bf16`: `bfloat16` 정밀도를 사용하도록 설정합니다. `bfloat16`은 FP32와 유사한 범위를 제공하면서 모리와 계산 효율성을 높입니다.

- `learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 용하여 안정적인 학습을 유도합니다.

- `max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 정하여 폭발적 그래디언트를 방지합니다.

- `warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다 학습의 안정성을 높이기 위해 사용됩니다.

- `lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"costant"`는 학습률을 일정하게 유지합니다.

- `push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니. `False`로 설정하면 업로드하지 않습니다.

- `remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다.`True`로 설정하면 메모리를 절약할 수 있습니다.

- `dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

- `report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn


In [14]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

- 라마 챗 템플릿

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations.<|eot_id|><|start_header_id|>user<|end_header_id|>

What can you help me with?<|eot_id|><|start_header_id|>assistant<|end_header_id|>|>

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 레이블 데이터를 초기화한 다음, 특정 토큰 패<|start_header_id|>assistantnt 이후부터 eot_idnd|>까지)을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [16]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 3234])
어텐션 마스크 형태: torch.Size([1, 3234])
레이블 형태: torch.Size([1, 3234])


In [17]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 115036, 99901, 18918, 82818, 120378, 43139, 109760, 19954, 111964, 110513, 109670, 382, 13447, 49531, 21028, 67890, 30426, 115790, 18359, 103386, 100968, 119978, 627, 16, 13, 109760, 54780, 115036, 99901, 18918, 82818, 120378, 43139, 111964, 16582, 119978, 627, 17, 13, 115036, 99901, 19954, 108838, 109842, 18359, 111964, 16582, 113348, 117193, 96677, 119978, 627, 18, 13, 109760, 19954, 102597, 108386, 13094, 115036, 99901, 19954, 47782, 115300, 115036, 99901, 102772, 330, 34983, 65895, 109760, 93, 19954, 102597, 109842, 13094, 120078, 1210, 103959, 35495, 111964, 16582, 119978, 627, 19, 13, 111964, 48936, 54718, 103966, 30381, 117294, 18918, 119884, 83290, 54535, 41953, 108520, 54535, 101353, 18359, 114839, 101528, 33390, 107333, 19954, 102722, 102657, 16969, 23955, 101711, 84734, 17835, 95713, 117294, 85721, 48424, 18918, 102484, 21121, 119978, 13, 96717, 18918, 105510, 27796, 103966, 30381, 54535, 41953, 10659

In [19]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 콘코바르는 결국 죽게 되는데, 그 곡절은 이러하다. 라긴의 왕 메스 게그러의 뇌를 굳힌 겻을 울라의 코날이 전리품으로 가지고 있었는데, 코나크타의 전사 케트 막 마가크가 이를 훔쳐갔다. 그리고 케트는 무릿매로 메스 게그러의 뇌를 던져 콘코바르의 머리를 맞추었고, 메스 게그러의 뇌가 콘코바르의 머리통 깊숙히 박혀 버렸다. 이 일이 일어난 곳은 우르카르(Urchair)의 발러 아흐(Baile Ath), 곧 오늘날의 웨스트미스 주 호르셀리프라고 한다. 콘코바르의 의사들은 이 이물질을 제거할 수 없었고, 상처를 봉합만 한 뒤 왕에게 흥분하지 않으면 생명을 유지할 수 있다고 말했다. 7년이 평화롭게 흘러간 뒤 콘코바르는 그리스도가 죽었다는 소식을 듣게 되어 분노했고, 뇌가 터져 죽었다. 머리가 터진 자리에서 뿜어져나온 피의 세례를 받은 결과 그는 기독교인이 되었고 그 영혼은 천국으로 갔다. 콘코바르의 죽음에 관한 이 기록은 매우 얄팍한 기독교화가 이루어져 있는데, 한편 노르드 신화의 

In [18]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [20]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
검색 결과에는 콘코바르가 왕이 된 이후 이멘마하에서 있었던 첫 번째 전투를 찾을 수 없습니다.<|eot_id|>


### input_ids와 labels는 어떻게 생성되는가?


LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.
사용자가 `안녕하세요, 오늘 날씨는 어떤가요?`라고 물었고,
모델은 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`라고 응답해야 한다고 합시다.

LLaMA 3에서는 다음과 같은 템플릿 구조를 사용합니다:

`<|begin_of_text|><|start_header_id|>user<|end_header_id|>
안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>`

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환됩니다. 예시로 단순화된 정수 시퀀스는 다음과 같다고 가정합니다:

`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`

여기서 모델이 예측해야 할 영역은 assistant의 응답 부분인
`안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다.
따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져
모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다.

이를 통해 모델은 불필요한 입력 부분을 학습하지 않고, assnt 응답 부분에만 집중할 수 있습니다.
"""


## 5. 학습

In [21]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [22]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.858300
20,0.634700
30,0.512500
40,0.490500
50,0.470700
60,0.428600
70,0.402700
80,0.477200
90,0.391100
100,0.341500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 6. 테스트 데이터 준비


실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|start_header_id|>assistant<|end_header_id|>\n'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [23]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [56]:
print(prompt_lst[700])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 세계 3위 유통업체인 테스코가 중국에서 9년간 진행해온 독자사업을 포기했다고 텔레그래프가 11일 보도했다.테스코는 대신 중국 국영 화룬(華潤)기업과 합작법인을 설립하기로 했다. 테스코는 중국 내 121개 매장을 이 합작법인에 넘기고 지분 20%를 받을 계획이다. 이번 합작은 양사 간 윈윈으로 보이지만 사실상 테스코가 중국 시장에서 백기를 든 것이라고 로이터통신이 지적했다. 테스코는 올 들어 중국 내 매출이 4.9% 하락하는 등 중국 시장 공략에 어려움을 겪어왔다. 이 회사는 올해 미국, 지난해에는 일본 시장에서 철수키로 결정했으며 당분간 모국인 영국 투자에 집중할 계획이다.화룬은 “합작법인을 통해 단일 브랜드로 대형마트와 슈퍼마켓, 편의점 등 다양한 사업을 벌일 것”이라고 밝혔다. 화룬그룹은 중국 본토와 홍콩에 매장 3000개를 운영하고 있는 중국 최대 유통업체다.중국에서 고전하고 있는 다국적 유통업체는 테스코만이 아니다. 올해 초에는 독일 유통업체 메트로가 중국 내 가전제품 사업부를 철수시켰고, 사무용품 

In [57]:
print(label_lst[700])


테스코는 수익성이 낮은 43개 점포를 폐쇄하고 출점 예정이었던 49개 점포의 출점을 취소하기로 결정했습니다. 따라서 테스코가 출점을 취소한 총 개수는 49개입니다 [[ref4]].


## 7. 파인튜닝 모델 테스트

In [26]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [61]:
peft_model_id = "llama-3-8b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [62]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [63]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [66]:
for prompt, label in zip(prompt_lst[700:703], label_lst[700:703]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
테스코가 출점을 취소한 총 개수는 49개입니다. 테스코는 수익성이 낮은 점포 43개를 폐쇄하고, 출점 예정이었던 49개 점포의 출점을 내지 않기로 결정했습니다 [[ref4]].
    label:

테스코는 수익성이 낮은 43개 점포를 폐쇄하고 출점 예정이었던 49개 점포의 출점을 취소하기로 결정했습니다. 따라서 테스코가 출점을 취소한 총 개수는 49개입니다 [[ref4]].
--------------------------------------------------
    response:
검찰 내부에서 더 선호하는 검사 드라마를 방영하는 방송국은 MBC입니다. SBS의 '펀치'보다 MBC의 '오만과 편견'에 대한 선호도가 높다고 언급되었습니다. '펀치'는 부패하고 탐욕이 강한 고위급 검사를 다루는 내용이 많아 부정적인 이미지를 부각시키는 반면, '오만과 편견'은 범죄와 치열하게 싸우는 긍정적인 모습을 보여주기 때문에 검찰 내부에서 더 선호된다고 설명되었습니다 [[ref2]].
    label:

검찰 내부에서 더 선호하는 검사 드라마를 방영하는 방송국은 MBC입니다. MBC의 드라마 '오만과 편견'이 검찰 내부에서 더 긍정적인 평가를 받고 있습니다. 이는 '펀치'가 검찰을 부정적으로 그리는 대목이 많은 반면, '오만과 편견'은 범죄와 치열하게 싸우는 비교적 긍정적인 모습을 보여주기 때문입니다. 검사들 사이에서는 '펀치'가 검찰에 대한 부정적인 사건을 짜깁기해 만든 것 같다는 의견이 있으며, '오만과 편견'은 현실과 동떨어진 느낌이 덜하다는 평가를 받고 있습니다 [[ref2]].
--------------------------------------------------
    response:
정부와 FTA 논의 계획 중인 국가들을 포함하는 대륙은 중미와 남미입니다. 정부는 중미 7개국(과테말라, 엘살바도르, 온두라스, 니카라과, 코스타리카, 파나마, 에콰도르)과 FTA 협상을 개시할 계획이며, 중미경제통합상설사무국(SIECA) 소속

## 8. 기본 모델 테스트

In [59]:
base_model_id = "NCSOFT/Llama-VARCO-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [60]:
for prompt, label in zip(prompt_lst[700:705], label_lst[700:705]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
테스코가 출점을 취소한 총 개수는 49개입니다.

문서4에 따르면 "수익성이 낮은 점포 43곳의 문을 닫고 출점 예정이었던 49개 점포를 내지 않기로 했다"라고 명시되어 있습니다.

따라서 출점을 취소한 총 개수는 43(폐쇄된 점포 수) + 49(출점 예정이었으나 취소된 점포 수) = 92개가 아니라, 오류로 인해 43과 49를 더한 대신 49만을 언급하면 정확한 답이 됩니다. 

정확한 답변: 49개
    label:

테스코는 수익성이 낮은 43개 점포를 폐쇄하고 출점 예정이었던 49개 점포의 출점을 취소하기로 결정했습니다. 따라서 테스코가 출점을 취소한 총 개수는 49개입니다 [[ref4]].
--------------------------------------------------
    response:
검찰 내부에서 더 선호하는 드라마에 대해 직접적으로 언급된 부분은 없습니다. 

하지만 문서2에 따르면 "검사 A씨는 ‘TV에 검사가 자주 등장하고 가족들이 ‘진짜로 저래’라고 물어보는 터라 관심을 안 가질 수가 없다’며 ‘검사나 수사관끼리 모이면 종종 가십성으로 검찰 드라마를 화제에 올린다’고 말했다."라는 내용이 있습니다. 

이는 검사들이 자신들의 직업에 대해 더 깊이 이해하고 공감할 수 있는 드라마를 선호할 수 있음을 시사합니다. 

하지만 특정 방송국이나 드라마를 선호하는 이유에 대해서는 명확히 언급되지 않았습니다. 

따라서 검찰 내부에서 가장 선호하는 방송국이나 드라마를 정확히 말하기는 어렵습니다. 

검사들이 선호하는 드라마의 선택 기준에 대해서는 개인의 취향, 가치관, 드라마의 내용 등 다양한 요인이 복합적으로 작용할 것으로 추측할 수 있겠습니다. 

더 구체적인 정보가 필요하다면 직접 검찰 관계자나 관련 전문가에게 조사를 해볼 필요가 있을 것 같습니다.
    label:

검찰 내부에서 더 선호하는 검사 드라마를 방영하는 방송국은 MBC입니다. MBC의 드라마 '오만과 편견'이 검찰 내부에서 더 긍정적인 